<a href="https://colab.research.google.com/github/fabiobento/dnn-course-2024-1/blob/main/00_course_folder/cert_prof_time_series/class_04/TS%20-%20W4%20-%2009%20-%20Manchas%20Solares%20com%20DNN%20(Laborat%C3%B3rio%202).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

adaptado de [Certificado Profissional Desenvolvedor do TensorFlow](https://www.coursera.org/professional-certificates/tensorflow-in-practice) de [Laurence Moroney](https://laurencemoroney.com/)

# Previsão de manchas solares com redes neurais (somente DNN)

Nos laboratórios restantes sobre séries temporais, você deixará de lado as séries temporais sintéticas e começará a criar modelos para dados do mundo real.

Em particular, você treinará no conjunto de dados [Sunspots](https://www.kaggle.com/datasets/robervalt/sunspots):
* um registro mensal do número de manchas solares de janeiro de 1749 a julho de 2018.

Primeiro, você criará uma rede neural profunda composta de camadas densas. Isso servirá como linha de base para que você possa compará-la com o próximo laboratório, no qual você usará uma arquitetura mais complexa.

Vamos começar!

## Importações
Você usará as mesmas importações de antes, com a adição do módulo [csv](https://docs.python.org/3/library/csv.html).

Você precisará dele para analisar o arquivo CSV que contém o conjunto de dados.

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import csv

In [ ]:
# Baixar arquivos adicionais para o laboratório
!wget https://github.com/fabiobento/dnn-course-2024-1/raw/main/00_course_folder/cert_prof_time_series/class_04/Sunspots.csv
!unzip -n -q Sunspots.csv

## Utilitários

Você só terá o conjunto de dados `plot_series()` aqui porque não precisa mais das funções de geração de dados sintéticos.

In [ ]:
def plot_series(x, y, format="-", start=0, end=None,
                title=None, xlabel=None, ylabel=None, legend=None ):
    """
    Visualiza dados de séries temporais

    Args:
      x (array of int) - contém valores para o eixo x
      y (matriz de int ou tupla de matrizes) - contém os valores para o eixo y
      format (string) - estilo de linha ao plotar o gráfico
      label (string) - etiqueta para a linha
      start (int) - primeiro passo de tempo a ser plotado
      end (int) - último passo de tempo a ser plotado
      title (string) - título do gráfico
      xlabel (string) - rótulo do eixo x
      ylabel (string) - rótulo do eixo y
      legend (lista de strings) - legenda para o gráfico
    """

    # Configurar as dimensões da figura do gráfico
    plt.figure(figsize=(10, 6))

  # Verificar se há mais de duas séries para plotar
    if type(y) is tuple:

      # Fazer um loop sobre os elementos y
      for y_curr in y:

        # Plote os valores x e y atuais
        plt.plot(x[start:end], y_curr[start:end], format)

    else:
      # Plote os valores x e y
      plt.plot(x[start:end], y[start:end], format)

    # Rotular o eixo x
    plt.xlabel(xlabel)

    # Rotular o eixo y
    plt.ylabel(ylabel)

    # Definir a legenda
    if legend:
      plt.legend(legend)

    # Definir o título
    plt.title(title)

    # Sobrepor uma grade no gráfico
    plt.grid(True)

    # Desenhe o gráfico na tela
    plt.show()

## Download e visualização do conjunto de dados

Agora você pode fazer o download do conjunto de dados e inspecionar o conteúdo.

O link na aula é do meu repositório, mas o Google hospeda o link abaixo.

In [ ]:
# Baixar o conjunto de dados
!wget https://storage.googleapis.com/tensorflow-1-public/course4/Sunspots.csv

Executando a célula abaixo, você verá que há apenas três colunas no conjunto de dados:
1. coluna sem título contendo o número do mês
2. Data que tem o formato `YYYY-MM-DD`
3. Número médio total de manchas solares

In [ ]:
# Visualizar o conjunto de dados
!head Sunspots.csv

Para este laboratório e o próximo, você precisará apenas do número do mês e do número total médio de manchas solares. Você carregará esses dados na memória e os converterá em matrizes que representam uma série temporal.

In [ ]:
# Inicializar listas
time_step = []
sunspots = []

# Abrir arquivo CSV
with open('./Sunspots.csv') as csvfile:

  # Inicializar o leitor
  reader = csv.reader(csvfile, delimiter=',')

  # Pular a primeira linha
  next(reader)

  # Acrescente o número da linha e de sunspot  às listas
  for row in reader:
    time_step.append(int(row[0]))
    sunspots.append(float(row[2]))

# Converter listas em matrizes numéricas
time = np.array(time_step)
series = np.array(sunspots)

# Visualizar os dados
plot_series(time, series, xlabel='Month', ylabel='Monthly Mean Total Sunspot Number')

## Dividir o conjunto de dados

Em seguida, você dividirá o conjunto de dados em conjuntos de treinamento e validação.

Há 3.235 pontos no conjunto de dados e você usará os primeiros 3.000 para treinamento.

In [ ]:
# Definir o tempo de divisão
split_time = 3000

# Obter o conjunto de trens
time_train = time[:split_time]
x_train = series[:split_time]

# Obter o conjunto de validação
time_valid = time[split_time:]
x_valid = series[split_time:]

## Preparar recursos e rótulos

Em seguida, você pode preparar as janelas do conjunto de dados como antes.

O tamanho da janela é definido como 30 pontos (igual a 2,5 anos), mas fique à vontade para alterá-lo posteriormente se quiser fazer experiências.

In [ ]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    """Gera janelas de conjunto de dados

    Args:
      series (matriz de float) - contém os valores da série temporal
      window_size (int) - o número de etapas de tempo a serem incluídas no recurso
      batch_size (int) - o tamanho do lote
      shuffle_buffer(int) - tamanho do buffer a ser usado para o método shuffle

    Retorna:
      dataset (TF Dataset) - Conjunto de dados TF contendo janelas de tempo
    """

    # Gerar um conjunto de dados TF a partir dos valores da série
    dataset = tf.data.Dataset.from_tensor_slices(series)

    # Janela de dados, mas só pega aqueles com o tamanho especificado
    dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)

    # Achatar as janelas, colocando seus elementos em um único lote
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))

    # Criar tuplas com recursos e rótulos
    dataset = dataset.map(lambda window: (window[:-1], window[-1]))

     # Embaralhar as janelas
    dataset = dataset.shuffle(shuffle_buffer)

    # Criar lotes de janelas
    dataset = dataset.batch(batch_size).prefetch(1)

    return dataset

In [ ]:
# Hiperpârametros
window_size = 30
batch_size = 32
shuffle_buffer_size = 1000

# Gerar as janelas do conjunto de dados
train_set = windowed_dataset(x_train, window_size, batch_size, shuffle_buffer_size)

## Construir o modelo

O modelo será uma rede densa de 3 camadas, conforme mostrado abaixo.

In [ ]:
# Construir o modelo
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(30, input_shape=[window_size], activation="relu"),
    tf.keras.layers.Dense(10, activation="relu"),
    tf.keras.layers.Dense(1)
])

# Imprimir o resumo do modelo
model.summary()

## Ajuste a taxa de aprendizado

Você pode escolher uma taxa de aprendizagem executando o mesmo código de agendamento de taxa de aprendizagem dos laboratórios anteriores.

In [ ]:
# Definir o agendador de taxa de aprendizado
lr_schedule = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-8 * 10**(epoch / 20))

# Inicializar o otimizador
optimizer = tf.keras.optimizers.SGD(momentum=0.9)

# Definir os parâmetros de treinamento
model.compile(loss=tf.keras.losses.Huber(), optimizer=optimizer)

# Treinar o modelo
history = model.fit(train_set, epochs=100, callbacks=[lr_schedule])

In [ ]:
# Definir a matriz de taxa de aprendizado
lrs = 1e-8 * (10 ** (np.arange(100) / 20))

# Definir o tamanho da figura
plt.figure(figsize=(10, 6))

# Definir a grade
plt.grid(True)

# Plotar a perda em escala logarítmica
plt.semilogx(lrs, history.history["loss"])

# Aumentar o tamanho dos tickmarks
plt.tick_params('both', length=10, width=1, which='both')

# Definir os limites do gráfico
plt.axis([1e-8, 1e-3, 0, 100])

## Treinar o modelo

Depois de escolher uma taxa de aprendizado, você pode reconstruir o modelo e iniciar o treinamento.

In [ ]:
# Redefinir estados gerados pelo Keras
tf.keras.backend.clear_session()

# Criar o modelo
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(30, input_shape=[window_size], activation="relu"),
    tf.keras.layers.Dense(10, activation="relu"),
    tf.keras.layers.Dense(1)
])

In [ ]:
# Definir a taxa de aprendizado
learning_rate = 2e-5

# Definir o otimizador
optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9)

# Definir os parâmetros de treinamento
model.compile(loss=tf.keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mae"])

# Treinar o modelo
history = model.fit(train_set,epochs=100)

## Previsão do modelo

Agora veja se o modelo gera bons resultados. Se você usou os parâmetros padrão deste notebook, deverá ver que as previsões seguem a forma do ground-truth com um MAE de cerca de 15.

In [ ]:
def model_forecast(model, series, window_size, batch_size):
    """Usa um modelo de entrada para gerar previsões em janelas de dados

    Args:
      model (TF Keras Model) - modelo que aceita janelas de dados
      series (array of float) - contém os valores da série temporal
      window_size (int) - o número de etapas de tempo a serem incluídas na janela
      batch_size (int) - o tamanho do lote

    Retorna:
      forecast (matriz numpy) - matriz que contém as previsões
    """

    # Gerar um conjunto de dados TF a partir dos valores da série
    dataset = tf.data.Dataset.from_tensor_slices(series)

    # Janela de dados, mas só pega aqueles com o tamanho especificado
    dataset = dataset.window(window_size, shift=1, drop_remainder=True)

    # Achatar as janelas, colocando seus elementos em um único loteh
    dataset = dataset.flat_map(lambda w: w.batch(window_size))

    # Criar lotes de janelas
    dataset = dataset.batch(batch_size).prefetch(1)

    # Obter previsões em todo o conjunto de dados
    forecast = model.predict(dataset)

    return forecast

In [ ]:
# Reduzir a série original
forecast_series = series[split_time-window_size:-1]

# Use a função auxiliar para gerar previsões
forecast = model_forecast(model, forecast_series, window_size, batch_size)

# Soltar eixo unidimensional
results = forecast.squeeze()

# Plotar os resultados
plot_series(time_valid, (x_valid, results))

In [ ]:
# Calcular o MAE
print(tf.keras.metrics.mean_absolute_error(x_valid, results).numpy())

## Resumo

Neste laboratório, você criou uma DNN relativamente simples para prever o número de manchas solares em um determinado mês.

Recomendamos que você ajuste os parâmetros ou treine por mais tempo e veja os melhores resultados que pode obter.

No próximo laboratório, você criará um modelo mais complexo e avaliará se a complexidade adicional se traduz em resultados melhores ou piores.